# Preparing data for ____

###### We are making a model to predict road accidents in NYC. 
To do this, we are going to use the following data 
- hourly traffic count on the bridges and tunnels of NYC 
- vehicle crashes in NYC 

We'll use the month, time of day and the vehicle count to predict how many accidents will happen at a future time and date for a given vehicle count.

We'll do this by... before you understand how we're doing this you must see-
### The datasets we are working with:

In [13]:
import pandas as pd

In [12]:
# hourly traffic raw data
dfHourlyTraffic = pd.read_csv('HourlyTraffic.csv')
dfHourlyTraffic.head()

,Plaza ID,Date,Hour,Direction,# Vehicles - ETC (E-ZPass),# Vehicles - Cash/VToll
0,1,11/28/2015,0,I,477,205
1,1,11/28/2015,0,O,486,252
2,1,11/28/2015,1,I,350,171
3,1,11/28/2015,1,O,307,182
4,1,11/28/2015,2,I,280,133


In [11]:
# vehicle crash data
dfVehicleCrashes = pd.read_csv('VehicleCrashes.csv' , low_memory=False)
dfVehicleCrashes.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,10/16/2018,0:00,BROOKLYN,11215,40.668990,-73.99641,"(40.66899, -73.99641)",15 STREET,2 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4001454,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,10/16/2018,0:00,MANHATTAN,10039,40.824196,-73.94224,"(40.824196, -73.94224)",NaN,NaN,85 BRADHURST AVENUE,...,NaN,NaN,NaN,NaN,4000711,Sedan,NaN,NaN,NaN,NaN
2,10/16/2018,0:00,QUEENS,11414,40.668860,-73.85491,"(40.66886, -73.85491)",80 STREET,149 AVENUE,NaN,...,NaN,NaN,NaN,NaN,4002838,NaN,NaN,NaN,NaN,NaN
3,10/16/2018,0:00,NaN,NaN,40.788692,-73.93787,"(40.788692, -73.93787)",EAST 106 STREET,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4001629,Sedan,Sedan,NaN,NaN,NaN
4,10/16/2018,0:00,NaN,NaN,40.757668,-73.95915,"(40.757668, -73.95915)",FDR DRIVE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4000768,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN


### Steps to solving the problem